In [1]:
from pyspark.sql import (
    functions as f,
    SparkSession,
    types as t
)

spark = SparkSession.builder.appName("logical_physical_play").getOrCreate()

In [2]:
file_path = "file:///home/jovyan/work/lorem_ipsum.txt"
df = spark.read.text(file_path)
df.explain(mode="formatted")

== Physical Plan ==
Scan text  (1)


(1) Scan text 
Output [1]: [value#0]
Batched: false
Location: InMemoryFileIndex [file:/home/jovyan/work/lorem_ipsum.txt]
ReadSchema: struct<value:string>




In [3]:
words = df.select(
    f.explode(
        f.split(df.value, ' ')).alias("word"))
word_counts = words.groupBy("word").count()
word_counts.explain(extended=True)

== Parsed Logical Plan ==
'Aggregate ['word], ['word, count(1) AS count#8L]
+- Project [word#3]
   +- Generate explode(split(value#0,  , -1)), false, [word#3]
      +- Relation [value#0] text

== Analyzed Logical Plan ==
word: string, count: bigint
Aggregate [word#3], [word#3, count(1) AS count#8L]
+- Project [word#3]
   +- Generate explode(split(value#0,  , -1)), false, [word#3]
      +- Relation [value#0] text

== Optimized Logical Plan ==
Aggregate [word#3], [word#3, count(1) AS count#8L]
+- Generate explode(split(value#0,  , -1)), [0], false, [word#3]
   +- Relation [value#0] text

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- HashAggregate(keys=[word#3], functions=[count(1)], output=[word#3, count#8L])
   +- Exchange hashpartitioning(word#3, 200), ENSURE_REQUIREMENTS, [plan_id=18]
      +- HashAggregate(keys=[word#3], functions=[partial_count(1)], output=[word#3, count#12L])
         +- Generate explode(split(value#0,  , -1)), false, [word#3]
            +- FileScan t

In [4]:
word_counts.explain(mode="formatted")

== Physical Plan ==
AdaptiveSparkPlan (6)
+- HashAggregate (5)
   +- Exchange (4)
      +- HashAggregate (3)
         +- Generate (2)
            +- Scan text  (1)


(1) Scan text 
Output [1]: [value#0]
Batched: false
Location: InMemoryFileIndex [file:/home/jovyan/work/lorem_ipsum.txt]
ReadSchema: struct<value:string>

(2) Generate
Input [1]: [value#0]
Arguments: explode(split(value#0,  , -1)), false, [word#3]

(3) HashAggregate
Input [1]: [word#3]
Keys [1]: [word#3]
Functions [1]: [partial_count(1)]
Aggregate Attributes [1]: [count#11L]
Results [2]: [word#3, count#12L]

(4) Exchange
Input [2]: [word#3, count#12L]
Arguments: hashpartitioning(word#3, 200), ENSURE_REQUIREMENTS, [plan_id=18]

(5) HashAggregate
Input [2]: [word#3, count#12L]
Keys [1]: [word#3]
Functions [1]: [count(1)]
Aggregate Attributes [1]: [count(1)#7L]
Results [2]: [word#3, count(1)#7L AS count#8L]

(6) AdaptiveSparkPlan
Output [2]: [word#3, count#8L]
Arguments: isFinalPlan=false


